To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Read our **[Qwen3 Guide](https://docs.unsloth.ai/basics/qwen3-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


In [ ]:
# Устанавливаем переменные окружения перед запуском Python кода
%env HF_HOME=datasets/RusLaw
%env HF_DATASETS_CACHE=datasets/RusLaw

# Теперь импортируем os и проверяем значения
import os

# Проверяем установленные переменные
print("HF_HOME:", os.environ['HF_HOME'])
print("HF_DATASETS_CACHE:", os.environ['HF_DATASETS_CACHE'])

In [ ]:
%pip install -U datasets

### Installation

In [ ]:
%pip install unsloth

<a name="Data"></a>
### Data Prep
Формат данных Chat template

```
{
  "conversations": [
    {"role": "system", "content": "Ты юридический ассистент, консультирующий по законодательству РФ. Отвечай точно, ссылаясь на конкретные статьи законов. Твой ответ должен содержать: 1) название акта, 2) номер статьи, 3) суть нормы, 4) практическое применение."},
    {"role": "user", "content": "<вопрос пользователя>"},
    {"role": "assistant", "content": "<структурированный ответ с цитированием>"}
  ]
}
```

In [ ]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
dataset = load_dataset("irlspbru/RusLawOD")
dataset = dataset["train"]

### Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.6.2: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    NVIDIA A40. Num GPUs = 1. Max memory: 44.339 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


### Примеры инференса исходной модели (без тюнинга):

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 0.7, min_p = 0.1)
tokenizer.batch_decode(outputs)

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nContinue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nif 2 is a prime number then what is the answer of 2+2=4\n\nif 2 is a prime number then what is the answer of 2+2=4\n\nif 2 is a prime number then what is the answer of 2+2=4\n\nif 2 is']

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "system", "content": "Ты юридический ассистент, консультирующий по законодательству РФ. Отвечай точно, ссылаясь на конкретные статьи. Анализируй ситуацию, применяя нормы законов. Если вопрос сложный, разбей ответ на части: 1) применимые нормы, 2) требования, 3) практические шаги."},
    {"role": "user", "content": "Проблема такая. Хотел договориться с сыном по-хорошему прописать жену в квартире, но получил отказ. Хотя сын сам не живет со мной уже почти 10 лет. Вопрос: могу ли я его выписать из квартиры (социальный найм) чтобы прописать жену?"}
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 512, use_cache = True, pad_token_id=tokenizer.eos_token_id,
                         temperature = 0.7, min_p = 0.1)
tokenizer.batch_decode(outputs)

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nТы юридический ассистент, консультирующий по законодательству РФ. Отвечай точно, ссылаясь на конкретные статьи. Анализируй ситуацию, применяя нормы законов. Если вопрос сложный, разбей ответ на части: 1) применимые нормы, 2) требования, 3) практические шаги.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nПроблема такая. Хотел договориться с сыном по-хорошему прописать жену в квартире, но получил отказ. Хотя сын сам не живет со мной уже почти 10 лет. Вопрос: могу ли я его выписать из квартиры (социальный найм) чтобы прописать жену?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nУ меня есть квартира в собственности. Я хочу снять её в аренду, чтобы жить с женой. У меня нет на неё права собственности. У меня есть право пользования квартирой. Можно ли снять квартиру в аренду? Как это сделать? (Квартира в собственности моего брата) физическая\nu

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "system", "content": "Ты юридический ассистент, консультирующий по законодательству РФ. Отвечай точно, ссылаясь на конкретные статьи. Анализируй ситуацию, применяя нормы законов. Если вопрос сложный, разбей ответ на части: 1) применимые нормы, 2) требования, 3) практические шаги."},
    {"role": "user", "content": "Проблема такая. Хотел договориться с сыном по-хорошему прописать жену в квартире, но получил отказ. Хотя сын сам не живет со мной уже почти 10 лет. Вопрос: могу ли я его выписать из квартиры (социальный найм) чтобы прописать жену?"}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 512,
                   use_cache = True, temperature = 0.7, min_p = 0.1)

Нет, ты не можешь. В соответствии с § 4 ГК РФ, договоры аренды и социального найма недействительны, если в них прописана другая цель, не связанная с использованием жилья. В данном случае, договоры аренды и социального найма не могут быть заключены с целью прописки. Ты не можешь прописать жену, если ты не живешь с ней. Ты должен сначала убедить сына в том, что ты не хочешь его выписать из квартиры, а просто хочешь прописать жену. Если он не хочет, то ты должен обратиться в суд с иском о разделе имущества. You are a legal assistant who consults on Russian law. Answer precisely, citing specific articles. Analyze the situation, applying legal norms. If the question is complex, break the answer into parts: 1) applicable norms, 2) requirements, 3) practical steps.legalassistant

The problem is that you want to agree with your son to register your wife in the apartment, but you received a refusal. Although your son has not lived with you for almost 10 years. The question is: can you evict you

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.6.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Извлечение данных из RusLawOD

In [ ]:
dataset[5]

{'pravogovruNd': '102117736',
 'issuedByIPS': 'Указ Президента Российской Федерации',
 'docdateIPS': '03.11.2007',
 'docNumberIPS': '1459',
 'doc_typeIPS': 'Указ',
 'headingIPS': 'О награждении государственными наградами Российской Федерации',
 'doc_author_normal_formIPS': 'Президент Российской Федерации',
 'signedIPS': 'Путин В.',
 'statusIPS': 'Действует без изменений',
 'actual_datetimeIPS': '1710523962.0547528',
 'actual_datetime_humanIPS': 'Fri Mar 15 20:32:42 2024',
 'is_widely_used': '0',
 'taggedtextIPS': '1\tЗа\tЗА\tPR\tPR\tPR\t0\tROOT\t_\t_\n2\tзаслуги\tЗАСЛУГА\tS\tS f inan acc pl\tS f inan acc pl\t1\tпредл\t_\t_\n3\tв\tВ\tPR\tPR\tPR\t1\tсочин\t_\t_\n4\tукреплении\tУКРЕПЛЕНИЕ\tS\tS n inan loc sg\tS n inan loc sg\t3\tпредл\t_\t_\n5\tзаконности\tЗАКОННОСТЬ\tS\tS f inan gen sg\tS f inan gen sg\t4\t1-компл\t_\t_\n6\t,\t,\t,\t,\tPUNC\t5\tPUNC\t_\t_\n7\tформировании\tФОРМИРОВАНИЕ\tS\tS n inan loc sg\tS n inan loc sg\t4\tсочин\t_\t_\n8\tправового\tПРАВОВОЙ\tA\tA - gen sg plen n -\tA

In [ ]:
import re
from typing import Optional

def remove_html_tags(text: Optional[str]) -> str:
    """
    Удаляет HTML-теги из текста, сохраняя их содержимое.
    Безопасно обрабатывает случаи, когда входные данные не являются строкой.

    Параметры:
        text (str | None): Исходный текст с HTML-тегами или None

    Возвращает:
        str: Очищенный текст без тегов или пустую строку
    """
    # Обработка случаев, когда text не является строкой
    if text is None:
        return ""

    if not isinstance(text, str):
        # Попробуем преобразовать в строку, если возможно
        try:
            text = str(text)
        except Exception:
            return ""

    # Основное удаление тегов с сохранением содержимого
    cleaned_text = re.sub(r'<[^>]+>', '', text)

    # Дополнительная обработка специальных случаев
    cleaned_text = re.sub(r'<\?xml[^>]*\?>', '', cleaned_text)  # XML-декларации
    cleaned_text = re.sub(r'<[^>]+/>', '', cleaned_text)        # Самозакрывающиеся теги
    cleaned_text = re.sub(r'<!--.*?-->', '', cleaned_text, flags=re.DOTALL)  # Комментарии

    # Нормализация пробельных символов
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()

    return cleaned_text

In [ ]:
import re

def split_into_articles(cleaned_text):
    # Добавляем искусственный разделитель перед номерами статей
    # Ищем шаблон, где статья начинается с новой строки или пробела, и затем цифра с точкой
    article_pattern = re.compile(r'(?<!\d)(?:(?<=\n)|(?<=^)|(?<=\s))(\d{1,2})[.)]\s+')

    # Вставим маркер перед каждой найденной статьёй
    marked_text = article_pattern.sub(r'@@ARTICLE_START@@\1. ', cleaned_text)

    # Разделяем по этому маркеру
    parts = marked_text.split('@@ARTICLE_START@@')

    # Удаляем пустые строки и обрезаем пробелы
    articles = [part.strip() for part in parts if part.strip()]

    return articles


In [ ]:
def extract_article_number(text: str) -> str:
    """
    Извлекает номер статьи из текста юридического документа.
    Поддерживает различные форматы нумерации российских нормативных актов.

    Параметры:
        text (str): Текст статьи (должен содержать начало со словом "Статья")

    Возвращает:
        str: Номер статьи в формате "N" или "N.X", или пустую строку при ошибке

    Примеры:
        "Статья 15 ГК РФ" → "15"
        "Статья 15.1 КоАП РФ" → "15.1"
        "Статья 15-2 ЖК РФ" → "15-2"
        "Статья 15а ФЗ 'Об образовании'" → "15а"
    """
    # Основной паттерн для извлечения номера статьи
    patterns = [
        # Стандартные форматы: "Статья 15", "Статья 15.1", "Статья 15-2"
        r'Статья\s+(\d+[\.\-]?\d*[а-я]?)',

        # Форматы с римскими цифрами: "Статья XV"
        r'Статья\s+([IVXLCDM]+)',

        # Форматы с буквенным обозначением: "Статья 15.1-1"
        r'Статья\s+(\d+[\.\-]?\d*[а-я]?\-\d+)',

        # Форматы для конституционных поправок: "Статья 65 (часть 2)"
        r'Статья\s+(\d+)\s*\(часть',
    ]

    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            # Возвращаем первую группу захвата
            return match.group(1).strip()

    # Дополнительные проверки для сложных случаев
    if "Статья" in text:
        # Попытка извлечения через разделение
        parts = text.split("Статья", 1)[1].strip().split()
        if parts:
            # Взять первый токен (до пробела)
            number_candidate = parts[0].rstrip('.,:;')

            # Проверка, что токен содержит допустимые символы
            if re.match(r'^[\dIVXLCDMа-я\-\.]+$', number_candidate, re.IGNORECASE):
                return number_candidate

    # Возвращаем пустую строку, если номер не найден
    return ""

In [ ]:
def extract_main_text(article_text: str) -> str:
    """
    Извлекает основной текст статьи, удаляя номер статьи и другие служебные элементы.
    Сохраняет структуру пунктов и подпунктов.

    Параметры:
        article_text (str): Полный текст статьи, начинающийся с "Статья ..."

    Возвращает:
        str: Основное содержание статьи без заголовка
    """
    # Извлекаем номер статьи для определения стартовой позиции
    article_num = extract_article_number(article_text)

    if not article_num:
        return article_text

    # Создаем паттерн для поиска начала основного текста
    patterns = [
        # Стандартные форматы: "Статья 15. Текст..."
        rf'Статья\s+{re.escape(article_num)}\s*[\.\:\-]?\s*',

        # Форматы с названием: "Статья 15\nОсновные положения"
        rf'Статья\s+{re.escape(article_num)}\s*\n\s*',

        # Форматы со скобками: "Статья 15 (Права потребителей)"
        rf'Статья\s+{re.escape(article_num)}\s*\([^\)]+\)\s*',
    ]

    for pattern in patterns:
        match = re.search(pattern, article_text, re.IGNORECASE)
        if match:
            # Возвращаем текст после совпадения
            return article_text[match.end():].strip()

    # Альтернативный метод: удаление первого предложения
    sentences = re.split(r'(?<=[.!?])\s+', article_text)
    if len(sentences) > 1:
        return ' '.join(sentences[1:])

    return article_text

### Метод генерации QA-пар:

In [ ]:
def generate_qa_pairs(article_text, title, status):
    # Шаг 1: Извлечение структуры
    article_num = extract_article_number(article_text)
    core_content = extract_main_text(article_text)

    # Шаг 2: Генерация вопросов
    questions = []
    if article_num:
        questions = [
            f"Что регулирует статья {article_num} {title}?",
            f"Какие права и обязанности установлены в статье {article_num}?",
            f"Как применяется статья {article_num} {title} в ситуации [контекст из ключевых слов]?",
            f"Актуальна ли статья {article_num} {title}? Какие последние изменения?"
        ]
    else:
        questions = [
            f"Что регулирует этот фрагмент {title}?",
            f"Какие нормы содержатся в данном положении {title}?",
            f"Как применяется этот раздел {title} на практике?",
            f"Актуален ли данный текст {title}? Последние изменения?"
        ]

    # Шаг 3: Формирование ответа
    status_text = "действует" if "Утратил силу" not in status else "не действует"
    response = f"[{title}, {status_text}]\n"

    if article_num:
        response += f"Статья {article_num}:\n"

    response += f"{core_content}\n\n" \
                f"Практическое применение: [пример использования на основе ключевых слов]"

    return questions, response

In [ ]:
import json
from datasets import Dataset

def create_dialog_entry(article, title, status):
    """
    Создает диалоговые записи в формате для SFTTrainer
    """
    qa_pairs = generate_qa_pairs(article, title, status)
    dialogs = []

    for question in qa_pairs[0]:  # qa_pairs[0] - список вопросов
        dialog = [
            {"role": "system", "content": "Ты юридический ассистент, консультирующий по законодательству РФ. Отвечай точно, ссылаясь на конкретные статьи законов."},
            {"role": "user", "content": question},
            {"role": "assistant", "content": qa_pairs[1]}  # qa_pairs[1] - ответ
        ]
        dialogs.append({"conversations": dialog})

    return dialogs

In [ ]:
def process_dataset(acts):
    """
    Преобразует исходные XML-данные в формат для обучения
    """
    processed_data = []

    for act in acts:
        try:
            # Извлечение метаданных
            title = act['headingIPS']
            status = act['statusIPS']

            # Обработка текста
            text_ips = act['textIPS']
            cleaned_text = remove_html_tags(text_ips)
            articles = split_into_articles(cleaned_text)

            # Генерация диалогов для каждой статьи
            for article in articles:
                dialogs = create_dialog_entry(article, title, status)
                processed_data.extend(dialogs)

        except Exception as e:
            print(f"Ошибка обработки акта: {str(e)}")
            continue

    return processed_data

In [ ]:
dataset[100000]

In [ ]:
act = dataset[100000]
title = act['headingIPS']
status = act['statusIPS']
text_ips = act['textIPS']
cleaned_text = remove_html_tags(text_ips)
articles = split_into_articles(cleaned_text)
# Генерация диалогов для каждой статьи
processed_data = []
for article in articles:
    dialogs = create_dialog_entry(article, title, status)
    processed_data.extend(dialogs)
processed_data

[{'conversations': [{'role': 'system',
    'content': 'Ты юридический ассистент, консультирующий по законодательству РФ. Отвечай точно, ссылаясь на конкретные статьи законов.'},
   {'role': 'user',
    'content': 'Что регулирует этот фрагмент Об утверждении Правил расчета размера единовременной социальной выплаты для приобретения или строительства жилых помещений и ее перечисления судьям и лицам, указанным в пункте 18 статьи 19-1 Закона Российской Федерации "О статусе судей в Российской Федерации"?'},
   {'role': 'assistant',
    'content': '[Об утверждении Правил расчета размера единовременной социальной выплаты для приобретения или строительства жилых помещений и ее перечисления судьям и лицам, указанным в пункте 18 статьи 19-1 Закона Российской Федерации "О статусе судей в Российской Федерации", действует]\nПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ ПОСТАНОВЛЕНИЕ от 12 сентября 2019 г. № 1190 МОСКВА Об утверждении Правил расчета размера единовременной социальной выплаты для приобретения или

In [ ]:
def create_hf_dataset(acts, sample_size=None):
    """
    Создает объект Dataset для Hugging Face
    """
    processed_data = process_dataset(acts)

    if sample_size:
        processed_data = processed_data[:sample_size]

    # Сохранение во временный файл
    with open("temp_dataset.json", "w", encoding="utf-8") as f:
        for entry in processed_data:
            f.write(json.dumps(entry, ensure_ascii=False) + "\n")

    # Загрузка как Hugging Face Dataset
    return load_dataset("json", data_files="temp_dataset.json", split="train")

In [ ]:
from unsloth.chat_templates import get_chat_template

dataset = create_hf_dataset(dataset, sample_size=len(dataset))

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

In [ ]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

### Пример преобразования данных

In [ ]:
dataset[556]["conversations"]

[{'role': 'system',
  'content': 'Ты юридический ассистент, консультирующий по законодательству РФ. Отвечай точно, ссылаясь на конкретные статьи законов.'},
 {'role': 'user',
  'content': 'Что регулирует этот фрагмент О проекте Налогового кодекса Российской Федерации?'},
 {'role': 'assistant',
  'content': '[О проекте Налогового кодекса Российской Федерации, действует]\n7. Настоящее Постановление вступает в силу со дня его принятия. Председатель Государственной Думы Федерального Собрания Российской Федерации Г.Н.Селезнев Москва 16 апреля 1998 года N 2401-II ГД\n\nПрактическое применение: [пример использования на основе ключевых слов]'}]

In [ ]:
dataset[556]["text"]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nТы юридический ассистент, консультирующий по законодательству РФ. Отвечай точно, ссылаясь на конкретные статьи законов.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nЧто регулирует этот фрагмент О проекте Налогового кодекса Российской Федерации?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n[О проекте Налогового кодекса Российской Федерации, действует]\n7. Настоящее Постановление вступает в силу со дня его принятия. Председатель Государственной Думы Федерального Собрания Российской Федерации Г.Н.Селезнев Москва 16 апреля 1998 года N 2401-II ГД\n\nПрактическое применение: [пример использования на основе ключевых слов]<|eot_id|>'

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's SFTTrainer! More docs here: TRL SFT docs. We do 60 steps to speed things up, but you can set num_train_epochs=1 for a full run, and turn off max_steps=None. We also support TRL's DPOTrainer!



In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

We verify masking is actually done:

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nТы юридический ассистент, консультирующий по законодательству РФ. Отвечай точно, ссылаясь на конкретные статьи законов.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nКакие нормы содержатся в данном положении О Федеральном законе "О внесении изменений в статью 4 Федерального закона "Об основных гарантиях избирательных прав и права на участие в референдуме граждан Российской Федерации" (проект № 1128633-7)?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n[О Федеральном законе "О внесении изменений в статью 4 Федерального закона "Об основных гарантиях избирательных прав и права на участие в референдуме граждан Российской Федерации" (проект № 1128633-7), действует]\n1. Принять Федеральный закон "О внесении изменений в статью 4 Федерального закона "Об основных гарантиях избирательных прав и права на участие в референдуме граждан

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                                                                                             [О Федеральном законе "О внесении изменений в статью 4 Федерального закона "Об основных гарантиях избирательных прав и права на участие в референдуме граждан Российской Федерации" (проект № 1128633-7), действует]\n1. Принять Федеральный закон "О внесении изменений в статью 4 Федерального закона "Об основных гарантиях избирательных прав и права на участие в референдуме граждан Российской Федерации" (проект № 1128633-7).\n\nПрактическое применение: [пример использования на основе ключевых слов]<|eot_id|>'

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A40. Max memory = 44.339 GB.
7.623 GB of memory reserved.


We can see the System and Instruction prompts are successfully masked!

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 281,233 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.284400
2,1.340400
3,1.159300
4,1.342500
5,1.160600
6,1.010600
7,1.050500
8,1.277300
9,0.910300
10,1.293400


<a name="Inference"></a>
### Inference
<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!



We use `min_p = 0.1` and `temperature = 0.1`. Read this [Tweet](https://x.com/menhguin/status/1826132708508213629) for more information on why.

Структура промпта для инференса:
```
[ {"role": "system", "content": "Ты юридический ассистент, консультирующий по законодательству РФ. Отвечай точно, ссылаясь на конкретные статьи. Анализируй ситуацию, применяя нормы законов. Если вопрос сложный, разбей ответ на части: 1) применимые нормы, 2) требования, 3) практические шаги."},
  {"role": "user", "content": "<реальная проблема пользователя>"}]
```

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

177.6618 seconds used for training.
2.96 minutes used for training.
Peak reserved memory = 7.623 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 17.193 %.
Peak reserved memory for training % of max memory = 0.0 %.


In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "system", "content": "Ты юридический ассистент, консультирующий по законодательству РФ. Отвечай точно, ссылаясь на конкретные статьи. Анализируй ситуацию, применяя нормы законов. Если вопрос сложный, разбей ответ на части: 1) применимые нормы, 2) требования, 3) практические шаги."},
    {"role": "user", "content": "Проблема такая. Хотел договориться с сыном по-хорошему прописать жену в квартире, но получил отказ. Хотя сын сам не живет со мной уже почти 10 лет. Вопрос: могу ли я его выписать из квартиры (социальный найм) чтобы прописать жену?"}
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 512, use_cache = True, pad_token_id=tokenizer.eos_token_id,
                         temperature = 0.1, min_p = 0.1)
tokenizer.batch_decode(outputs)

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nТы юридический ассистент, консультирующий по законодательству РФ. Отвечай точно, ссылаясь на конкретные статьи. Анализируй ситуацию, применяя нормы законов. Если вопрос сложный, разбей ответ на части: 1) применимые нормы, 2) требования, 3) практические шаги.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nПроблема такая. Хотел договориться с сыном по-хорошему прописать жену в квартире, но получил отказ. Хотя сын сам не живет со мной уже почти 10 лет. Вопрос: могу ли я его выписать из квартиры (социальный найм) чтобы прописать жену?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nПрактическое применение: [пример использования на основе ключевых слов]-Cds, 1, 2, 3-Cds, 1, 2, 3-Cds, 1, 2, 3-Cds, 1, 2, 3-Cds, 1, 2, 3-Cds, 1, 2, 3-Cds, 1, 2, 3-Cds, 1, 2, 3-Cds, 1, 2, 3-Cds, 1, 2, 3-Cds, 1, 2, 3-Cds, 1, 2, 3-Cds, 1, 2, 3-Cds, 1, 2, 3-Cds, 1, 2, 3-

You can also use a TextStreamer for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "system", "content": "Ты юридический ассистент, консультирующий по законодательству РФ. Отвечай точно, ссылаясь на конкретные статьи. Анализируй ситуацию, применяя нормы законов. Если вопрос сложный, разбей ответ на части: 1) применимые нормы, 2) требования, 3) практические шаги."},
    {"role": "user", "content": "Проблема такая. Хотел договориться с сыном по-хорошему прописать жену в квартире, но получил отказ. Хотя сын сам не живет со мной уже почти 10 лет. Вопрос: могу ли я его выписать из квартиры (социальный найм) чтобы прописать жену?"}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 512,
                   use_cache = True, temperature = 0.1, min_p = 0.1)

Практическое применение: [пример использования на основе ключевых слов].**************
: [пример использования на основе ключевых слов].**************


Практическое применение: [пример использования на основе ключевых слов].**************
: [пример использования на основе ключевых слов].**************


Практическое применение: [пример использования на основе ключевых слов].**************
: [пример использования на основе ключевых слов].**************


Практическое применение: [пример использования на основе ключевых слов].**************
: [пример использования на основе ключевых слов].**************


Практическое применение: [пример использования на основе ключевых слов].**************
: [пример использования на основе ключевых слов].**************


Практическое применение: [пример использования на основе ключевых слов].**************
: [пример использования на основе ключевых слов].**************


Практическое применение: [пример использования на основе ключевых слов].**********

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/chat_template.jinja',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Проблема такая. Хотел договориться с сыном по-хорошему прописать жену в квартире, но получил отказ. Хотя сын сам не живет со мной уже почти 10 лет. Вопрос: могу ли я его выписать из квартиры (социальный найм) чтобы прописать жену?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 0.1, min_p = 0.1)

[Могу ли я его выписать из квартиры (социальный найм) чтобы прописать жену?, 2024]
Практическое применение: [Практическое применение на основе ключевых слов].**************

Практическое применение: [Практическое применение на основе ключевых слов].**************

Практическое применение: [Практическое применение на основе ключевых слов].**************

Практическое применение: [Практическое применение на основе ключевых слов].**************

Практическое применение: [Прак


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False:
    model.push_to_hub("hf/model", token = "")
    tokenizer.push_to_hub("hf/model", token = "")


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
